<a href="https://colab.research.google.com/github/bacchus00/limpieza-aportes-servel/blob/main/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

work_dir = '/content/drive/MyDrive/Tesis/Trabajo Dirigido/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install unidecode
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=a0f3c6fd192df56093780bb768700cc75699076d43152ab06c18919dd03a9e23
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [5]:
import pandas as pd
import re
import numpy as np
import hashlib
import swifter

from unidecode import unidecode

# Importing

In [6]:
# importing data
raw_data = pd.read_csv(work_dir + 'data/' + 'aportes-raw.csv')

<ipython-input-6-daa1e2b29157>:2: DtypeWarning: Columns (1,2,4,5,9,11,12,17,18,23,24,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(work_dir + 'data/' + 'aportes-raw.csv')


In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488432 entries, 0 to 488431
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Archivo                    488432 non-null  object 
 1   RUTAportante               344231 non-null  object 
 2   DVAportante                228466 non-null  object 
 3   Aportante                  442385 non-null  object 
 4   RUTCandidato               481383 non-null  object 
 5   DVCandidato                397180 non-null  object 
 6   Candidato                  486498 non-null  object 
 7   Afiliacion                 480705 non-null  object 
 8   RUTPartido                 1705 non-null    float64
 9   DVPartido                  1705 non-null    object 
 10  Partido                    438152 non-null  object 
 11  Pacto                      225156 non-null  object 
 12  Subpacto                   147134 non-null  object 
 13  Fecha                      42

In [160]:
# general cleansing
df = raw_data.copy()

df = df.applymap(lambda x: unidecode(' '.join(x.split()).title()) if type(x)==str else x)
df['Archivo'] = df['Archivo'].map(lambda x: x.lower())

df = df.replace({'-': np.nan, '':np.nan,})

df = df.drop('TipoPlanilla', axis=1)

# Rows With Errors

In [142]:
df[df['Eleccion'].isna() | (df['Eleccion']=='Ingresos Rechazados')]

,Archivo,RUTAportante,DVAportante,Aportante,RUTCandidato,DVCandidato,Candidato,Afiliacion,RUTPartido,DVPartido,...,DescripcionTipoDocumento,GlosaDocumento,Eleccion,Comuna,Region,Distrito,Circunscripcion,CircunscripcionProvincial,CircunscripcionSenatorial,TerritorioElectoral
29960,2004-alcalde-consejal-ingresos-partidos.xls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112544,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112545,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112546,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Ingresos Rechazados,NaN,*,NaN,NaN,NaN,NaN,NaN


In [161]:
drop_rows = df.index[
    df['Eleccion'].isna() |
    (df['Eleccion']=='Ingresos Rechazados')
]

df = df.drop(drop_rows)

# Monto

In [162]:
# replacing 0 for nans
df['Monto'] = df['Monto'].replace({0: np.nan})

In [145]:
df['Monto'].describe()

count    4.801260e+05
mean     9.519577e+05
std      1.490500e+07
min      1.000000e+00
25%      2.500000e+04
50%      1.000000e+05
75%      3.500000e+05
max      3.578743e+09
Name: Monto, dtype: float64

# Destino Aporte

In [163]:
# new column 'TipoBeneficiario' from filename
df['TipoBeneficiario'] = df['Archivo'].map(lambda x: 'Partido' if 'partido' in x else 'Candidato')

In [147]:
df['TipoBeneficiario'].value_counts(dropna=False)

Candidato    484074
Partido        4354
Name: TipoBeneficiario, dtype: int64

# Eleccion

In [164]:
# new column for election year from filename year
df['AnioEleccion'] = df['Archivo'].map(lambda x: x[:4])

In [149]:
df['AnioEleccion'].value_counts(dropna=False)

2021    214206
2016    111536
2017     46715
2008     35626
2004     29960
2012     27655
2013      8909
2005      5708
2009      4840
2020      3273
Name: AnioEleccion, dtype: int64

In [165]:
# renaming elections from filename information
df['Eleccion'] = df['Eleccion'].replace(to_replace={'Segunda Votacion Presidente':'Presidente'})

In [166]:
df['Eleccion'].value_counts(dropna=False)

Concejal                      230719
Alcalde                        91313
Diputado                       63644
Presidente                     29546
Convencional Constituyente     27856
Consejero Regional             22680
Senador                        16964
Gobernador Regional             5706
Name: Eleccion, dtype: int64

In [167]:
# new column for election typefrom filename year
df['TipoEleccion'] = df['Archivo'].map(lambda x: 'Primarias' if 'primaria' in x else 'Regular')
df['TipoEleccion'] = df.swifter.apply(lambda x: 'Segunda' if '2da' in x['Archivo'] else x['TipoEleccion'], axis=1)

Pandas Apply:   0%|          | 0/488428 [00:00<?, ?it/s]

In [168]:
df['TipoEleccion'].value_counts(dropna=False)

Regular      467317
Segunda       11551
Primarias      9560
Name: TipoEleccion, dtype: int64

In [ ]:
df[['AnioEleccion','Eleccion','TipoEleccion']].value_counts(dropna=False).reset_index().sort_values(['AnioEleccion','Eleccion','TipoEleccion'])

# Territorio Electoral

In [171]:
pd.set_option('display.max_rows', None)

In [181]:
territorios = [
  'TerritorioElectoral',
  'Circunscripcion',
  'Region',
  'CircunscripcionSenatorial',
  'Distrito',
  'CircunscripcionProvincial',
  'Comuna',

]

In [202]:
replace_territorio = {
    'Nacional':np.nan,
    'Pais':np.nan,
    'Circunscripcion Senatorial':np.nan,
    'Region':np.nan,
    'Partido': np.nan,
    'Partido Politico': np.nan
}

df['TerritorioElectoral'] = df['TerritorioElectoral'].replace(replace_territorio)

In [200]:
replace_region = {
    'Nacional': np.nan,
    'Partido': np.nan,
    'Partido Politico': np.nan
}

df['Region'] = df['Region'].replace(replace_region)

In [205]:
# valid election territories
territorios_validos = [
  'Region',
  'CircunscripcionSenatorial',
  'Distrito',
  'CircunscripcionProvincial',
  'Comuna',
]

# fills election territory columns according to the election
def replace_territorio(x): #refactorizable
  region = np.nan
  cir_sen = np.nan
  distrito = np.nan
  cir_prov = np.nan
  comuna = np.nan

  if x['Eleccion']=='Gobernador Regional':
    if x['Region']==x['Region']:
      region = x['Region']
    else:
      region = x['TerritorioElectoral']

  elif x['Eleccion']=='Consejero Regional':
    if x['CircunscripcionProvincial']==x['CircunscripcionProvincial']:
      cir_prov = x['CircunscripcionProvincial']
    elif x['Circunscripcion']==x['Circunscripcion']:
      cir_prov = x['Circunscripcion']
    else:
      cir_prov = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Senador':
    if x['CircunscripcionSenatorial']==x['CircunscripcionSenatorial']:
      cir_sen = x['CircunscripcionSenatorial']
    elif x['Circunscripcion']==x['Circunscripcion']:
      cir_sen = x['Circunscripcion']
    else:
      cir_sen = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Diputado' or x['Eleccion']=='Convencional Constituyente':
    if x['Distrito']==x['Distrito']:
      distrito = x['Distrito']
    else:
      distrito = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Alcalde' or x['Eleccion']=='Concejal':
    if x['Comuna']==x['Comuna']:
      comuna = x['Comuna']
    else:
      comuna = x['TerritorioElectoral']
    region = x['Region']

  return pd.Series([region, cir_sen, distrito, cir_prov, comuna], index=territorios_validos)

df[territorios_validos] = df.swifter.apply(replace_territorio, axis=1)

Pandas Apply:   0%|          | 0/488428 [00:00<?, ?it/s]

In [210]:
# importing electoral division
div_parl = pd.read_csv(work_dir+'data/'+'division-electoral-parlamentaria-chile-1989-2018.csv')

# creating comuna-ditrito and comuna cicunscripcion-electoral dictionaries
comuna_distrito = pd.Series(div_parl['Distrito'].values,index=div_parl['Comuna']).to_dict()
comuna_cir_sen = pd.Series(div_parl['Circunscripcion'].values,index=div_parl['Comuna']).to_dict()

#replacing territorie values
region_replace = {
    '13': 'Region Metropolitana De Santiago',
    '6': "Region Del Libertador General Bernardo O'Higgins",
    '9': 'Region De La Araucania',
    '16': 'Region De Nuble',
    '11': 'Region De Aysen Del General Carlos Ibanez Del Campo',
    '7': 'Region Del Maule',
    '3': 'Region De Atacama',
    '2': 'Region De Antofagasta',
    '5': 'Region De Valparaiso',
    '1': 'Region De Tarapaca',
    '8': 'Region Del Biobio',
    '14': 'Region De Los Rios',
    '4': 'Region De Coquimbo',
    '15': 'Region De Arica Y Parinacota',
    '10': 'Region De Los Lagos',
    '12': 'Region De Magallanes Y De La Antartica Chilena',
    'Partido Politico': np.nan,
    'Region': np.nan

}

cir_sen_replace = {
    'Circunscripcion Senatorial': np.nan,
    'CircunscripcionSenatorial': np.nan,
    'Aisen':'Aysen',
}

distrito_replace = {
    'Distrito': np.nan,
    'Aisen':'Aysen',
    'Llay-Llay': 'Llaillay',
    'O Higgins':"O'Higgins",
}

cir_prov_replace = {
    'Circunscripcion Provincial': np.nan,
    'Bio Bio': 'Biobio',
    'Cachapoal I': 'Cachapoal 1',
    'Cachapoal Ii': 'Cachapoal 2',
    'Cautin I': 'Cautin 1',
    'Cautin Ii': 'Cautin 2',
    'Concepcion I': 'Concepcion 1',
    'Concepcion Ii': 'Concepcion 2',
    'Concepcion Iii': 'Concepcion 3',
    'Santiago I': 'Santiago 1',
    'Santiago Ii': 'Santiago 2',
    'Santiago Iii': 'Santiago 3',
    'Santiago Iv': 'Santiago 4',
    'Santiago V': 'Santiago 5',
    'Santiago Vi': 'Santiago 6',
    'Valparaiso I': 'Valparaiso 1',
    'Valparaiso Ii': 'Valparaiso 2',
}

comuna_replace = {
    'Alto Bio Bio': 'Alto Biobio',
    'O Higgins': "O'Higgins",
    'O`Higgins': "O'Higgins",
    'Partido Politico': np.nan,
}

df['Region'] = df['Region'].map(lambda x: str(x) if x==x else x)
df['Region'] = df['Region'].replace(region_replace)

df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].map(lambda x: str(x).replace('.0','').replace('Circunscripcion Senatorial Ndeg','') if x==x else x)
df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].replace(cir_sen_replace)
df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].map(lambda x: str(comuna_cir_sen[x]) if x in comuna_cir_sen.keys() else x)

df['Distrito'] = df['Distrito'].map(lambda x: str(x).replace('.0','').replace('Distrito Ndeg','').replace('Distrito No ','') if x==x else x)
df['Distrito'] = df['Distrito'].replace(distrito_replace)
df['Distrito'] = df['Distrito'].map(lambda x: str(comuna_distrito[x]) if x in comuna_distrito.keys() else x)

df['CircunscripcionProvincial'] = df['CircunscripcionProvincial'].replace(cir_prov_replace)

df['Region'] = df['Region'].replace(region_replace)

df['Comuna'] = df['Comuna'].replace(comuna_replace)

In [212]:
df['Region'].value_counts(dropna=False)

NaN                                                    137212
Region Metropolitana De Santiago                        96697
Region Del Biobio                                       37701
Region De Valparaiso                                    35500
Region Del Maule                                        31206
Region Del Libertador General Bernardo O'Higgins        26802
Region De La Araucania                                  25206
Region De Los Lagos                                     20216
Region De Tarapaca                                      16087
Region De Coquimbo                                      13426
Region De Los Rios                                       9358
Region De Atacama                                        8360
Region De Antofagasta                                    7996
Region De Nuble                                          7209
Region De Magallanes Y De La Antartica Chilena           5480
Region De Aysen Del General Carlos Ibanez Del Campo      5128
Region D

In [213]:
df['CircunscripcionSenatorial'].value_counts(dropna=False)

NaN    471863
1        6130
10       2092
7        1401
15       1313
3         942
6         730
8         719
12        629
5         604
13        430
18        375
16        345
14        252
9         121
17        111
4         101
11         97
2          95
19         78
Name: CircunscripcionSenatorial, dtype: int64

In [ ]:
df['Distrito'].value_counts(dropna=False)

In [ ]:
df['CircunscripcionProvincial'].value_counts(dropna=False)

In [ ]:
df['Comuna'].value_counts(dropna=False)

In [221]:
# new columns TerritorioElectoral name of smallest territory column corresponding to election
eleccion_territorio_replace = {
  'Presidente': 'Pais',
  'Gobernador Regional': 'Region',
  'Consejero Regional': 'CircunscripcionProvincial',
  'Senador': 'CircunscripcionSenatorial',
  'Diputado': 'Distrito',
  'Convencional Constituyente': 'Distrito',
  'Alcalde': 'Comuna',
  'Concejal': 'Comuna',
}

df['TerritorioElectoral'] = df['Eleccion'].map(lambda x: eleccion_territorio_replace[x])

In [223]:
df['TerritorioElectoral'].value_counts(dropna=False)

Comuna                       322032
Distrito                      91500
Pais                          29546
CircunscripcionProvincial     22680
CircunscripcionSenatorial     16964
Region                         5706
Name: TerritorioElectoral, dtype: int64

# Afiliacion

In [224]:
# correcting afiliation status of candidates

archivo_afiliacion = {
  '2005-presidente-2da-ingresos-candidatos.xls': 'Afiliado',
  '2009-presidente-2da-ingresos-candidatos.xls': 'Afiliado',
  '2017-presidente-2da-ingresos-candidatos.xlsx': 'Independiente En Pacto'
}

candidato_prim_afiliacion = {
  'Daniel Jadue Jadue': 'Afiliado',
  'Gabriel Boric Font': 'Afiliado',
  'Ignacio Briones Rojas': 'Afiliado',
  'Joaquin Lavin Infante': 'Afiliado',
  'Mario Desbordes Jimenez': 'Afiliado',
  'Sebastian Sichel Ramirez': 'Independiente Asociado',
}

afiliacion_replace = {
  'Independiente En Pacto': 'Independiente Asociado',
  'Independiente En Subpacto': 'Independiente Asociado',
  'Indpendiente De Pacto': 'Independiente Asociado',
  'Independiente Asociado': 'Independiente Asociado',
  'Candidatura Independiente': 'Independiente No Asociado',
  'Independiente Fuera De Pacto': 'Independiente No Asociado',
}

df['Afiliacion'] = df.apply(lambda x: archivo_afiliacion[x['Archivo']] if x['Archivo'] in archivo_afiliacion else x['Afiliacion'], axis=1)
df['Afiliacion'] = df.apply(
    lambda x: candidato_prim_afiliacion[x['Candidato']] if x['Archivo']=='2021-presidente-primarias-ingresos-candidatos.xlsx' else x['Afiliacion'],
    axis=1
)
df['Afiliacion'] = df['Afiliacion'].replace(afiliacion_replace)

In [225]:
df['Afiliacion'].value_counts(dropna=False)

Afiliado                     324949
Independiente Asociado       130370
Independiente No Asociado     28755
NaN                            4354
Name: Afiliacion, dtype: int64

# Partido, RUTPartido y DVPartido

In [ ]:
# combines a rows rut and dv values into one
def rut_dv_merge(x, rut_col, dv_col):
  if x[rut_col]==x[rut_col]: # if not nan or
    if not '-' in str(x[rut_col]): # if not already in the right format
      if x[dv_col]==x[dv_col]: # if dv not nan
        return (str(x[rut_col]) + '-' + str(x[dv_col])).replace('.0','')
  else:
    return np.nan
  return str(x[rut_col]).replace('.0','')

# combines dataframe's rut and dv columns and restricts ruts to valid lengths
def rut_formatter(df, rut_col, dv_col):
  # RUT and DV merging
  df[rut_col] = df.apply(lambda x: rut_dv_merge(x, rut_col, dv_col), axis=1)
  df = df.drop(dv_col, axis=1)
  # Restricting to valid lengths
  df[rut_col] = df[rut_col].apply(lambda x: x if x==x and (len(str(x))==9 or len(str(x))==10) else np.nan)
  return df

# updates dataframe's rows with the same rut to have the same name
def same_rut_same_name(df, rut_col, name_col, update_dict={}, exclude=[], only_include=[]):
  dict_rut_name = df[~df[name_col].isna() & ~df[name_col].isna()][[rut_col,name_col]].groupby(rut_col).agg(lambda x: max(x, key=len)).to_dict()[name_col] # keeping longest name
  dict_rut_name.update(update_dict)
  if exclude:
    dict_rut_name = {k: v for k,v in dict_rut_name.items() if k not in exclude}
  if only_include:
    dict_rut_name = {k: v for k,v in dict_rut_name.items() if k in only_include}
  df[name_col] = df.apply(lambda x: dict_rut_name[x[rut_col]] if (x[rut_col]==x[rut_col] and x[rut_col] in dict_rut_name) else x[name_col], axis=1)
  return df

# updates dataframe's rows with the same name to have the same rut
def same_name_same_rut(df, rut_col, name_col, update_dict={}, exclude=[], only_include=[]):
  dict_name_rut = df[~df[name_col].isna() & ~df[rut_col].isna()][[name_col,rut_col]].groupby(name_col).max().to_dict()[rut_col] # keeping highest rut
  dict_name_rut.update(update_dict)
  if exclude:
    dict_name_rut = {k: v for k,v in dict_name_rut.items() if k not in exclude}
  if only_include:
    dict_name_rut = {k: v for k,v in dict_name_rut.items() if k in only_include}
  df[rut_col] = df.apply(lambda x: dict_name_rut[x[name_col]] if (x[name_col]==x[name_col] and x[name_col] in dict_name_rut) else x[rut_col], axis=1)
  return df


In [ ]:
# RUT correction
df = rut_formatter(df, 'RUTPartido', 'DVPartido')

In [ ]:
# party renaming
rename_party = {
    'Partido Evolucion Politica': 'Evolucion Politica',

    'Partido Radical De Chile': 'Partido Radical',
    'Partido Radical Socialdemocrata': 'Partido Radical',

    'Independiente': np.nan,
    'Independientes': np.nan,

    'Partido Ciudadanos': 'Ciudadanos',

    'Partido Ecologista Verde Del Norte': 'Partido Ecologista Verde',
    'Partido Ecologista': 'Partido Ecologista Verde',

    'Partido De Izquierda Ciudadana De Chile': 'Partido Izquierda Ciudadana De Chile',
    'Partido De Izquierda Ciudadana De Chile -Muni 2012 Y Parl 2013': 'Partido Izquierda Ciudadana De Chile',

    'Partido Progresista De Chile': 'Partido Progresista',

    'Partido De Los Trabajadores Revolucionarios': 'Partido De Trabajadores Revolucionarios',

    'Partido Igualdad': 'Igualdad'

}
df['Partido'].replace(to_replace=rename_party, inplace=True)

In [ ]:
df = same_rut_same_name(df, 'RUTPartido', 'Partido')
df = same_name_same_rut(df, 'RUTPartido', 'Partido')

In [ ]:
df[['Partido','RUTPartido']].value_counts(dropna=False)

Partido                                                  RUTPartido
NaN                                                      NaN           132461
Union Democrata Independiente                            71552600-K     52965
Renovacion Nacional                                      71395000-9     47386
Partido Democrata Cristiano                              71468400-0     43665
Partido Por La Democracia                                71455400-K     35788
                                                                        ...  
Fuerza Del Norte                                         NaN                5
Independientes De Nuble Por La Nueva Constitucion (D19)  NaN                5
No_Vigente                                               NaN                5
Cabildos Autoconvocados (D2)                             NaN                2
Por La Integracion Regional                              65092861-K         1
Length: 138, dtype: int64

In [ ]:
pr = df[['Partido','RUTPartido']].dropna()
rut_party_dict = pd.Series(pr['Partido'].values,index=pr['RUTPartido']).to_dict()
party_rut_dict = pd.Series(pr['RUTPartido'].values,index=pr['Partido']).to_dict()

In [ ]:
rut_party_dict.keys()

dict_keys(['71468400-0', '71701800-1', '71395000-9', '71706400-3', '72551500-6', '71552600-K', '71447800-1', '71455400-K', '65037533-5', '65166343-1', '65171967-4', '65119848-8', '65176888-8', '65107248-4', '65114322-5', '65108245-5', '65114239-3', '65102519-2', '65123086-1', '65194007-9', '65112171-K', '53325352-0', '65144125-0', '65165842-K', '65111743-7', '65172582-8', '65092861-K', '53325947-2', '65167417-4', '65171766-3', '65193238-6', '77150977-0', '53335163-8', '65188975-8', '65202859-4', '53335692-3'])

# Candidato, RUTCandidato and DVCandidato

In [ ]:
# correcting ruts
df = rut_formatter(df, 'RUTCandidato', 'DVCandidato')

In [ ]:
# setting same names for people with same ruts, prioritizing post-2016 records
dict_rut_candidato_2016 = df[~df['Candidato'].isna() & ~df['Candidato'].isna() & (df['AnioEleccion'].astype(int)>2016)][['RUTCandidato','Candidato']].groupby('RUTCandidato').agg(lambda x: max(x, key=len)).to_dict()['Candidato'] # keeping longest name
df = same_rut_same_name(df, 'RUTCandidato', 'Candidato', update_dict=dict_rut_candidato_2016)

In [ ]:
#candidatos_many_ruts = list(df[['Candidato', 'RUTCandidato']].dropna().drop_duplicates().groupby('Candidato').filter(lambda x: len(x) > 1)['Candidato'].unique())
#df = same_name_same_rut(df, 'RUTCandidato', 'Candidato', exclude=candidatos_many_ruts)
df['RUTCandidato'] = df.apply(lambda x: party_rut_dict[x['Candidato']] if (x['RUTCandidato']!=x['RUTCandidato'] and x['Candidato'] in party_rut_dict) else x['RUTCandidato'], axis=1)

In [ ]:
df[['Candidato', 'RUTCandidato']].sample(5)

,Candidato,RUTCandidato
231560,Jorge Soria Quiroga,3519419-3
106519,Mauricio Antonio Ojeda Rebolledo,15256333-7
344170,Carolina Leitao Alvarez-Salamanca,12488981-2
268613,Gabriel Silber Romo,8779559-4
116966,Guillermo Pedro Cejas Valenzuela,11816437-7


In [ ]:
cr = df[['Candidato','RUTCandidato']].dropna()
rut_candidato_dict = pd.Series(cr['Candidato'].values,index=cr['RUTCandidato']).to_dict()

In [ ]:
# removing parties from candidates
df.loc[df['RUTCandidato'].isin(rut_party_dict.keys()), ['RUTCandidato', 'Candidato']] = [np.nan, np.nan]

# Aportante, RUTAportante and DVAportante

In [ ]:
# correcting ruts
df = rut_formatter(df, 'RUTAportante', 'DVAportante')

In [ ]:
# setting same names for people with same ruts, prioritizing post-2016 records and including party names
dict_rut_aportante_2016 = df[~df['Aportante'].isna() & ~df['Aportante'].isna() & (df['AnioEleccion'].astype(int)>2016)][['RUTAportante','Aportante']].groupby('RUTAportante').agg(lambda x: max(x, key=len)).to_dict()['Aportante'] # keeping longest name
dict_rut_aportante_2016.update(rut_party_dict)
df = same_rut_same_name(df, 'RUTAportante', 'Aportante', update_dict=dict_rut_aportante_2016)

In [ ]:
#aportantes_many_ruts = list(df[['Aportante', 'RUTAportante']].dropna().drop_duplicates().groupby('Aportante').filter(lambda x: len(x) > 1)['Aportante'].unique())
#df = same_name_same_rut(df, 'RUTAportante', 'Aportante', exclude=aportantes_many_ruts)
df['RUTAportante'] = df.apply(lambda x: party_rut_dict[x['Aportante']] if (x['RUTAportante']!=x['RUTAportante'] and x['Aportante'] in party_rut_dict) else x['RUTAportante'], axis=1)

In [ ]:
df[['Aportante', 'RUTAportante']].dropna().sample(5)

,Aportante,RUTAportante
238222,Helvia Molina Vasquez,7064938-1
41513,Eugenio Carreno,15875104-K
216364,Luis Carrasco Nunez,13950274-4
134689,Com Y Servicios Aldotorres Benzi Eirl,76806510-1
304947,I Nterradio Tv Spa,76906541-5


# Fecha

In [226]:
# date format transformation
ddSmmSyy = r'\d\d/\d\d/\d\d'
ddSmmSyyyy = r'\d\d/\d\d/\d\d\d\d'
ddHmmHyy = r'\d\d-\d\d-\d\d'
ddHmmHyyyy = r'\d\d-\d\d-\d\d\d\d'
yyyyHmmHdd = r'\d\d\d\d-\d\d-\d\d'
yyyyHmmHdd_time = r'\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d'

def date_trasformer(a_date):
  if re.fullmatch(ddHmmHyy, a_date) or re.fullmatch(ddSmmSyy, a_date):
    return '20'+a_date[6:8]+'-'+a_date[3:5]+'-'+a_date[0:2]
  elif re.fullmatch(ddSmmSyyyy, a_date) or re.fullmatch(ddHmmHyyyy, a_date):
    return a_date[6:10]+'-'+a_date[3:5]+'-'+a_date[0:2]
  elif re.fullmatch(yyyyHmmHdd, a_date) or re.fullmatch(yyyyHmmHdd_time, a_date):
    return a_date[0:4]+'-'+a_date[5:7]+'-'+a_date[8:10]
  else:
    return a_date

# turns everything to YYYY-MM-DD (ISO 8601)
df['Fecha'] = df['Fecha'].map(lambda x: date_trasformer(x) if x==x else x)

In [233]:
df[df['Fecha']=='1004-10-31']

,Archivo,RUTAportante,DVAportante,Aportante,RUTCandidato,DVCandidato,Candidato,Afiliacion,RUTPartido,DVPartido,...,Comuna,Region,Distrito,Circunscripcion,CircunscripcionProvincial,CircunscripcionSenatorial,TerritorioElectoral,TipoBeneficiario,AnioEleccion,TipoEleccion
26766,2004-alcalde-consejal-ingresos-candidatos.xls,11564481-5,NaN,Claudio Sepulveda Segal,11564481-5,NaN,Claudio Sepulveda,Independiente No Asociado,NaN,NaN,...,Colbun,NaN,NaN,NaN,NaN,NaN,Comuna,Candidato,2004,Regular


In [232]:
df['Fecha'].dropna().astype('datetime64[D]').describe()

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1004-10-31 00:00:00 present at position 274

# OrigenAporte

In [ ]:
tipo_cuenta = [
    'Anticipo Financiamiento Fiscal', # fiscal
    'Aporte Anonimo', # anonimo
    'Aporte Con Publicidad', # con publicidad, puede cambiar
    'Aporte Estimado En Especies (Mantencion Y Colaciones)', # depende
    'Aporte Estimado Por Propaganda En Espacio Privado', # depende
    'Aporte Estimado Por Propaganda En Medios De Comunicacion Digital', # depende
    'Aporte Estimado Por Propaganda En Radioemisoras', # depende
    'Aporte Estimado Por Propaganda Escrita', # depende
    'Aporte Estimado Por Servicios Profesionales De Un Tercero', # depende
    'Aporte Estimado Por Uso De Propiedad De Un Tercero', # depende
    'Aporte Estimado Por Uso De Un Vehiculo De Un Tercero', # depende
    'Aporte Patrimonial', # depende
    'Aporte Personal Del Candidato', # propio
    'Aporte Personal Del Candidato (Vehiculo, Propiedades Entre Otros)', # propio
    'Aporte Por Trabajo Voluntario', # depende
    'Aporte Por Usufructo', # depende
    'Aporte Propio', # propio
    'Aporte Propio En Efectivo Del Candidato', # propio
    'Aporte Propio En Especies O Servicios', # propio
    'Aporte Propio Por Imputacion De Reembolso A Candidatos', # propio
    'Aporte Propio Por Imputacion De Reembolso A Instituciones Financieras', # propio
    'Aporte Publico', # publico, puede cambiar
    'Aporte Publico Partido Politico', # partido
    'Aporte Reservado', # reservado
    'Aporte Trabajo Voluntario', # depende
    'Aportes Anonimos', # anonimo
    'Aportes Con Publicidad', # con publicidad, puede cambiar
    'Aportes De Partido Politico', # partido
    'Aportes De Personas Naturales', # depende
    'Aportes En Dinero De Partidos Politicos', # partido
    'Aportes En Dinero Recibido Por Candidato De Un Partido Politico', # partido
    'Aportes En Especies O Servicios De Partidos Politicos', # partido
    'Aportes En Especies O Servicios De Terceros', # depende
    'Aportes En Especies Recibido Por Candidato De Un Partido Politico', # partido
    'Aportes Menores Sin Publicidad', # sin publicidad
    'Aportes Partido Politico', # partido
    'Aportes Partidos Politicos', # partido
    'Aportes Privados De Caracter Publico', # depende
    'Aportes Privados De Caracter Publico De Partidos Politicos', # partido
    'Aportes Propios De Partido Politico', # propio
    'Aportes Reservado', # reservado
    'Aportes Sin Publicidad', # sin publicidad
    'Cesion De Derecho A Reembolso', # depende
    'Comodato', # depende
    'Credito Institucion Financiera', # banco
    'Creditos Con Instituciones Financieras', # banco
    'Creditos Con Mandato', # banco
    'Donaciones', # depende
    'Financiamiento A Candidatos Independientes', # fiscal
    'Financiamiento A Partidos Politicos', # fiscal
    'Imputacion De Gasto Electoral De Primarias', # fiscal
    'Imputacion De Gasto Presunto De Partidos Politicos', # fiscal
    'Imputacion Gasto Presunto', # fiscal
    'Imputacion Gasto Primarias', # fiscal
    'Imputacion Saldo De Financiamiento Primarias', # fiscal
    'Intereses Credito Institucion Financiera', # banco
    'Intereses Devengados Por Creditos Con Instituciones Financieras', # banco
    'Intereses Por Credito Con Institucion Financiera', # banco
    'Mutuos', # depende
    'No Definido', # depende
    'Otros Actos O Contratos A Titulo Gratuito'# depende,
    'Reembolso De Gastos Electorales', # fiscal
    'Reembolso Facturas Cedidas Partido Politico', # fiscal
    'Reembolso Solicitado Candidato O Partido', # fiscal
    'Reembolso Solicitado Por Cesion De Derecho A Partido Politico', # fiscal
    'Reembolsos Facturas Endosadas', # fiscal
    'Reembolsos Solicitado Por Endosos', # fiscal
    'Usufructo'# depende
]

In [ ]:
#? 'Cesion De Derecho A Reembolso', 'No Definido',

anonimo = [ # se mantiene
    'Aporte Anonimo',
    'Aportes Anonimos',
]

publico = [
    'Aporte Publico'
]

reservado = [ # se mantiene
    'Aporte Reservado',
    'Aportes Reservado',
]

sin_publicidad = [ # se mantiene
    'Aportes Menores Sin Publicidad',
    'Aportes Sin Publicidad',
]

con_publicidad = [
    'Aporte Con Publicidad',
    'Aportes Con Publicidad',
]

fiscal = [ # del estado, se mantiene
    'Anticipo Financiamiento Fiscal',
    'Financiamiento A Candidatos Independientes',
    'Financiamiento A Partidos Politicos',

    'Imputacion De Gasto Electoral De Primarias',
    'Imputacion De Gasto Presunto De Partidos Politicos',
    'Imputacion Gasto Presunto',
    'Imputacion Gasto Primarias',
    'Imputacion Saldo De Financiamiento Primarias',

    'Reembolso De Gastos Electorales',
    'Reembolso Facturas Cedidas Partido Politico',
    'Reembolso Solicitado Candidato O Partido',
    'Reembolso Solicitado Por Cesion De Derecho A Partido Politico',
    'Reembolsos Facturas Endosadas',
    'Reembolsos Solicitado Por Endosos',
]

banco = [ # de una institucion financiera, se mantiene
    'Credito Institucion Financiera',
    'Creditos Con Instituciones Financieras',
    'Creditos Con Mandato',
    'Intereses Credito Institucion Financiera',
    'Intereses Devengados Por Creditos Con Instituciones Financieras',
    'Intereses Por Credito Con Institucion Financiera',
]

propio = [ # propio del candidato o del partido a si mismo, se mantiene
    'Aporte Personal Del Candidato',
    'Aporte Personal Del Candidato (Vehiculo, Propiedades Entre Otros)',
    'Aporte Propio',
    'Aporte Propio En Efectivo Del Candidato',
    'Aporte Propio En Especies O Servicios',
    'Aporte Propio Por Imputacion De Reembolso A Candidatos',
    'Aporte Propio Por Imputacion De Reembolso A Instituciones Financieras',
    'Aportes Propios De Partido Politico',
]

partido = [ # de un partido no a si mismo, se mantiene
    'Aporte Publico Partido Politico',
    'Aportes De Partido Politico',
    'Aportes En Dinero De Partidos Politicos',
    'Aportes En Dinero Recibido Por Candidato De Un Partido Politico',
    'Aportes En Especies O Servicios De Partidos Politicos',
    'Aportes En Especies Recibido Por Candidato De Un Partido Politico',
    'Aportes Partido Politico',
    'Aportes Partidos Politicos',
    'Aportes Privados De Caracter Publico De Partidos Politicos',
    'Reembolso Facturas Cedidas Partido Politico',
]

depende = [ # depende de quien es el aportante, aportes estimados
    'Aporte Estimado En Especies (Mantencion Y Colaciones)',
    'Aporte Estimado Por Propaganda En Espacio Privado',
    'Aporte Estimado Por Propaganda En Medios De Comunicacion Digital',
    'Aporte Estimado Por Propaganda En Radioemisoras',
    'Aporte Estimado Por Propaganda Escrita',
    'Aporte Estimado Por Servicios Profesionales De Un Tercero',
    'Aporte Estimado Por Uso De Propiedad De Un Tercero',
    'Aporte Estimado Por Uso De Un Vehiculo De Un Tercero',
    'Aporte Patrimonial',
    'Aporte Por Trabajo Voluntario',
    'Aporte Por Usufructo',
    'Aporte Trabajo Voluntario',
    'Aportes Con Publicidad',
    'Aportes De Personas Naturales',
    'Aportes En Especies O Servicios De Terceros',
    'Aportes Privados De Caracter Publico',
    'Comodato',
    'Donaciones',
    'Mutuos',
    'No Definido',
    'Otros Actos O Contratos A Titulo Gratuito',
    'Usufructo',
    'Cesion De Derecho A Reembolso'
]

state_inst = [
    '60504000-4',
    '60805000-0',
]

banks = [
]

se_mantienen = anonimo+reservado+sin_publicidad+fiscal+banco

def type_by_desc(x):
  if x['DescripcionTipoCuenta'] in anonimo:
    return 'Anonimo'
  if x['DescripcionTipoCuenta'] in publico:
    return 'Publico'
  if x['DescripcionTipoCuenta'] in reservado:
    return 'Reservado'
  if x['DescripcionTipoCuenta'] in sin_publicidad:
    return 'Sin Publicidad'
  if x['DescripcionTipoCuenta'] in con_publicidad:
    return 'Con Publicidad'
  if x['DescripcionTipoCuenta'] in propio:
    return 'Propio'
  if x['DescripcionTipoCuenta'] in partido:
    return 'Partido'
  if x['DescripcionTipoCuenta'] in fiscal:
    return 'Fiscal'
  if x['DescripcionTipoCuenta'] in banco:
    return 'Banco'
  else:
    return x['TipoAporte']

def type_by_agents(x):
  if x['RUTCandidato']==x['RUTAportante'] or (x['TipoIngreso']=='Partido' and x['RUTPartido']==x['RUTAportante']):
    return 'Propio'
  if x['RUTAportante'] in rut_party_dict.keys():
    if x['TipoIngreso']=='Candidato' or (x['TipoIngreso']=='Partido' and x['RUTAportante']!=x['RUTPartido']):
      return 'Partido'
  if x['RUTAportante'] in state_inst:
    return 'Fiscal'
  if x['RUTAportante'] in banks:
    return 'Banco'
  else:
    return x['TipoAporte']


def type_by_elim(x):
  if x['DescripcionTipoCuenta'] in depende:
    if int(x['AnioEleccion'])<2016:
      return 'Publico' if x['RUTAportante']==x['RUTAportante'] else 'Anonimo'
    else:
      return 'Con Publicidad' if x['RUTAportante']==x['RUTAportante'] else 'Sin Publicidad'
  else:
    return x['TipoAporte']

df['TipoAporte'] = np.nan

df['TipoAporte'] = df.apply(type_by_desc, axis=1)
df['TipoAporte'] = df.apply(lambda x: type_by_agents(x) if not x['DescripcionTipoCuenta'] in se_mantienen else x['TipoAporte'], axis=1)
df['TipoAporte'] = df.apply(lambda x: type_by_elim(x) if x['TipoAporte']!=x['TipoAporte'] else x['TipoAporte'], axis=1)

In [ ]:
cols = [
    'Archivo',
    'RUTAportante',
    'Aportante',
    'RUTCandidato',
    'Candidato',
    'Afiliacion',
    'RUTPartido',
    'Partido',
    'TipoAporte',
    'TipoAporte2',
    'TipoAporte3',
    'TipoCuenta', 'DescripcionTipoCuenta', 'EstadoCuenta',
    'NumeroDocumento', 'TipoDocumento', 'DescripcionTipoDocumento',
    'GlosaDocumento',]
uwu = df[df['TipoAporte']!=df['TipoAporte2']].dropna(subset=['TipoAporte', 'TipoAporte2'])[cols]
display(uwu)
uwu[['TipoAporte','TipoAporte2']].value_counts()

,Archivo,RUTAportante,Aportante,RUTCandidato,Candidato,Afiliacion,RUTPartido,Partido,TipoAporte,TipoAporte2,TipoAporte3,TipoCuenta,DescripcionTipoCuenta,EstadoCuenta,NumeroDocumento,TipoDocumento,DescripcionTipoDocumento,GlosaDocumento
39,2004-alcalde-consejal-ingresos-candidatos.xls,71706400-3,Partido Socialista De Chile,1721224-9,Carlos Yusta Rojas,Afiliado,71706400-3,Partido Socialista De Chile,Publico,Partido,Partido,140.0,Aporte Publico,NaN,28.0,Oi,NaN,Primera Remesa Aporte Partido Socialista De Chile
41,2004-alcalde-consejal-ingresos-candidatos.xls,71706400-3,Partido Socialista De Chile,1721224-9,Carlos Yusta Rojas,Afiliado,71706400-3,Partido Socialista De Chile,Publico,Partido,Partido,140.0,Aporte Publico,NaN,44.0,Oi,NaN,Segundo Aporte Partido Socialista De Chile
48,2004-alcalde-consejal-ingresos-candidatos.xls,72551500-6,Partido Radical,1779542-2,Jorge Salas Waak,Afiliado,72551500-6,Partido Radical,Publico,Partido,Partido,140.0,Aporte Publico,NaN,64.0,Cd,Certificado De Donacion,Cheque 73 5260486
86,2004-alcalde-consejal-ingresos-candidatos.xls,71468400-0,Partido Democrata Cristiano,1923810-5,Leocan Portus Govinden,Afiliado,71468400-0,Partido Democrata Cristiano,Publico,Partido,Partido,140.0,Aporte Publico,NaN,140.0,Cd,Certificado De Donacion,Aporte Publico
139,2004-alcalde-consejal-ingresos-candidatos.xls,71706400-3,Partido Socialista De Chile,2127488-7,Pedro Medina Sanchez,Afiliado,71706400-3,Partido Socialista De Chile,Publico,Partido,Partido,140.0,Aporte Publico,NaN,600.0,Oi,NaN,Aporte Partido Socialista De Chile Anexo 600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488415,2021-presidente-senadores-diputados-core-ingre...,60504000-4,Tesoreria General De La Republica,NaN,NaN,NaN,71706400-3,Partido Socialista De Chile,Partido,Fiscal,Fiscal,116.0,Aportes Partidos Politicos,NaN,1412431,Cb,Cartola Bancaria,Transferencia Desde Cuenta Corriente Cores
488416,2021-presidente-senadores-diputados-core-ingre...,60504000-4,Tesoreria General De La Republica,NaN,NaN,NaN,71706400-3,Partido Socialista De Chile,Partido,Fiscal,Fiscal,116.0,Aportes Partidos Politicos,NaN,1412431,Cb,Cartola Bancaria,Transfefencia Desde Cuenta Corriente Cores
488417,2021-presidente-senadores-diputados-core-ingre...,60504000-4,Tesoreria General De La Republica,NaN,NaN,NaN,71706400-3,Partido Socialista De Chile,Partido,Fiscal,Fiscal,116.0,Aportes Partidos Politicos,NaN,1412431,Cb,Cartola Bancaria,Transferencia Desde Cuenta Corriente Cores
488418,2021-presidente-senadores-diputados-core-ingre...,60504000-4,Tesoreria General De La Republica,NaN,NaN,NaN,71706400-3,Partido Socialista De Chile,Partido,Fiscal,Fiscal,116.0,Aportes Partidos Politicos,NaN,1412431,Cb,Cartola Bancaria,Transferencia Desde Cuenta Corriente Cores


TipoAporte      TipoAporte2
Con Publicidad  Partido        5382
Publico         Partido        2208
                Propio          274
Partido         Propio          202
Propio          Partido         104
Con Publicidad  Propio           78
Partido         Fiscal           26
Propio          Fiscal           16
Publico         Fiscal            4
dtype: int64

In [ ]:
iwi = df[df['TipoAporte3'].isna()][cols]
display(iwi)
iwi.DescripcionTipoCuenta.value_counts()

,Archivo,RUTAportante,Aportante,RUTCandidato,Candidato,Afiliacion,RUTPartido,Partido,TipoAporte,TipoAporte2,TipoAporte3,TipoCuenta,DescripcionTipoCuenta,EstadoCuenta,NumeroDocumento,TipoDocumento,DescripcionTipoDocumento,GlosaDocumento


Series([], Name: DescripcionTipoCuenta, dtype: int64)

In [ ]:
iwi.Aportante.unique()

array([], dtype=object)

# Aportante anonimization with md5 encription (not used)

In [ ]:
# applying md5 to ruts (except to parties and candidates)
def aux_fun_md5(x):
  if x == x:
    if (x in dict_candidate_rut.values()) or (x in dict_party_rut.values()):
      return x
    else:
      return hashlib.md5(str(x).encode()).hexdigest()
  else:
    return x

raw_data['RUTAportante'] = raw_data['RUTAportante'].apply(aux_fun_md5)

In [ ]:
# cheking number of unique ruts after md5 to verify absence of hash collsions
len(raw_data['RUTAportante'].unique())

In [ ]:
# cheking values after transformations
raw_data['RUTAportante'].value_counts()

# Exporting

In [ ]:
# column Re-ordering
df = df[[
    'Aportante',
    'RUTAportante',

    'Candidato',
    'RUTCandidato',
    'Afiliacion',

    'Partido',
    'RUTPartido',
    'Pacto',
    'Subpacto',

    'Monto',

    'AnioEleccion',
    'Eleccion',
    'TipoEleccion',

    'TerritorioElectoral',
    'Region',
    'CircunscripcionSenatorial',
    'Distrito',
    'CircunscripcionProvincial',
    'Comuna',

    'Fecha',
    'TipoAporte',
    'TipoIngreso',
    'TipoCuenta',
    'DescripcionTipoCuenta',
    'EstadoCuenta',
    'NumeroDocumento',
    'TipoDocumento',
    'DescripcionTipoDocumento',
    'GlosaDocumento',

    'Archivo'
]]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488428 entries, 0 to 488431
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Aportante                  421487 non-null  object 
 1   RUTAportante               285015 non-null  object 
 2   Candidato                  484345 non-null  object 
 3   RUTCandidato               480406 non-null  object 
 4   Afiliacion                 484074 non-null  object 
 5   Partido                    355967 non-null  object 
 6   RUTPartido                 344018 non-null  object 
 7   Pacto                      225156 non-null  object 
 8   Subpacto                   147134 non-null  object 
 9   Monto                      480126 non-null  float64
 10  AnioEleccion               488428 non-null  object 
 11  Eleccion                   488428 non-null  object 
 12  TipoEleccion               488428 non-null  object 
 13  TerritorioElectoral        48

In [ ]:
df.nunique()

Aportante                    138608
RUTAportante                 142249
Candidato                     40163
RUTCandidato                  40467
Afiliacion                        3
Partido                         137
RUTPartido                       36
Pacto                            53
Subpacto                         52
Monto                         65536
AnioEleccion                     10
Eleccion                          8
TipoEleccion                      3
TerritorioElectoral               6
Region                           16
CircunscripcionSenatorial        19
Distrito                         60
CircunscripcionProvincial        68
Comuna                          348
Fecha                          2353
TipoAporte                        9
TipoIngreso                       2
TipoCuenta                       45
DescripcionTipoCuenta            68
EstadoCuenta                      2
NumeroDocumento              148532
TipoDocumento                    27
DescripcionTipoDocumento    

In [ ]:
# exporting resulting data to csv
df.to_csv(work_dir + 'data/' + 'aportes.csv', index=False)